#### PLEASE NOTE
As this file contains lots of graphics, the file sizes exceeds more than 150MB.

So this file is only prioviding the code without any outputs. 

Please calculate the outputs on your own: `Kernel --> Restart & Run All`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from datetime import date, time, datetime, timedelta

import folium
from folium import plugins
from folium.plugins import HeatMap

import math

import random

# Color list
import matplotlib.colors as col

In [ ]:
boston = pd.read_csv("boston_2016.csv")
boston.head()

## Visualizing the location of the stations to get more insights

### First Step: Get coordinates of stations

As you can see at the open dataset of BlueBikes [here](https://s3.amazonaws.com/hubway-data/index.html), BlueBikes (originally Hubway) has done a relaunch in March 2018. Because of this, there were several data sets avaliable which contain the coordinates of stations. **However, no dataset did cover all stations we have in our provided dataset by the lecturer**. Thus, we need first to merge two potential files together to get a broader dataset for the `station <-> coordinates` mapping.

In [ ]:
geo_location_1 = pd.read_csv("previous_Hubway_Stations_as_of_July_2017.csv")
geo_location_2 = pd.read_csv("Hubway_Stations_2011_2016.csv")

def dataFrameContainsEntry(df, name):
    """This method returns false, if the given name does never appear in the given dataset."""
    return len(df[df["Station"]==name]) > 0

#check whether geo_location_1 already contains some names
for index, row in geo_location_2.iterrows():
    if dataFrameContainsEntry(geo_location_1, row["Station"]):
        geo_location_2.drop(index=index, inplace=True)

# Adding both datasets into one dataframe and dropping unnecessary values
geo_location = pd.concat([geo_location_1, geo_location_2])
geo_location.drop(labels=["Station ID", "Municipality", "publiclyExposed", "# of Docks"], axis=1, inplace=True)
geo_location.head(5)

### Second Step: Map coordinates to `boston` dataset

In [ ]:
def merge():
    #merge for start_station
    merged_df = pd.merge(boston, geo_location, how="left", left_on="start_station_name", right_on="Station")
    merged_df.drop(labels="Station", axis=1, inplace=True)
    merged_df.rename(columns={'Latitude':'Latitude_Start', 'Longitude': 'Longitude_Start'}, inplace=True)
    
    #merge for end_station
    merged_df = pd.merge(merged_df, geo_location, how="left", left_on="end_station_name", right_on="Station")
    merged_df.drop(labels="Station", axis=1, inplace=True)
    merged_df.rename(columns={'Latitude':'Latitude_End', 'Longitude': 'Longitude_End'}, inplace=True)
    return merged_df

    
merged_df = merge()
merged_df.head(5)    

### Third Step: Check new dataset for any invalid entries

In [ ]:
missing_stations = merged_df[merged_df.isnull().any(1)]
missing_stations.head(5)

Alright, there are some stations that did not get any coordinates yet. Ivestigating further.

In [ ]:
def getMissingStartStations():
    missing_stations = merged_df[merged_df["Latitude_Start"].isnull()] #return any row where some column has NaN value
    missing_stations_grouped = missing_stations.groupby("start_station_name").nunique()
    return missing_stations_grouped

def getMissingEndStations():
    missing_stations = merged_df[merged_df["Latitude_End"].isnull()] #return any row where some column has NaN value
    missing_stations_grouped = missing_stations.groupby("end_station_name").nunique()
    return missing_stations_grouped

def getMissingStations():
    missing_stations_grouped_start = getMissingStartStations()
    missing_stations_grouped_end = getMissingEndStations()

    missing_stations_grouped_start.reset_index(inplace=True)
    missing_stations_grouped_start.drop(columns=["end_time", "start_station_id", "end_station_id", 
                                                 "end_station_name", "bike_id", "user_type", 
                                                 "Latitude_Start", "Longitude_Start", "Latitude_End", "Longitude_End"], inplace=True)

    missing_stations_grouped_end.reset_index(inplace=True)
    missing_stations_grouped_end.drop(columns=["start_time", "start_station_id", "end_station_id", 
                                               "start_station_name", "bike_id", "user_type", 
                                               "Latitude_Start", "Longitude_Start", "Latitude_End", "Longitude_End"], inplace=True)

    missing_stations_total = pd.merge(missing_stations_grouped_start, missing_stations_grouped_end, how='outer', left_on="start_station_name", right_on="end_station_name")
    missing_stations_total.drop(columns="end_station_name", inplace=True)
    missing_stations_total.rename(columns={'start_station_name': 'station_name', 
                                           'start_time': 'Count of trips started at this station', 
                                           'end_time': 'Count of trips ended at this station'}, inplace=True)
    return missing_stations_total

In [ ]:
getMissingStations()
#Display stations which have NaN values and how often these stations were used in the dataset.

**Argh, although we already have used two different datasets, there are still 9 stations without coordinates. Most of them are even quite frequently used, so we cannot just drop them (e. g. `TD Garden - Causeway at Portal Park #1`)**. Need to investigate manually using more datasets provided by BlueBikes [here](https://s3.amazonaws.com/hubway-data/index.html). <br>

We used the following process:
1. As the [datasets provided by BlueBikes](https://s3.amazonaws.com/hubway-data/index.html) contain trip data for every month **WITH right coordinates**, we look up in which month the station occur in the dataset (use the method `getStartTime` below). We only focus here on missing `start_stations` because as shown above in the table the missing end and start stations does not differ.
1. Then go to [BlueBikes](https://s3.amazonaws.com/hubway-data/index.html) and download the dataset for the specific month.
1. Then look up in the dataset for the station which coordinates are provided.
1. Then create a new dataframe `geo_location_3` containing the missing coordinates.
1. Finally, add these new dataframe `geo_location_3` to the dataframe `geo_location`. 


In [ ]:
def getStartTime(station_name):
    df = boston[boston["start_station_name"]==station_name]
    return station_name + ": " + df["start_time"].head(1).values #return first value found
    
for missing_station in getMissingStations()["station_name"]:    
    print(getStartTime(missing_station))
    
# Getting for every missing station the first start_time where the station does occur in the dataset    

In [ ]:
# These values are taken manually from the BlueBikes website
dict_1 = {'Station': "18 Dorrance Warehouse", 'Latitude': 42.387151, 'Longitude': -71.075978}
dict_2 = {'Station': "TD Garden - Causeway at Portal Park #1", 'Latitude': 42.365942, 'Longitude': -71.060515}
 # This is no copy-error:
 # "TD Garden - Causeway at Portal Park #1" and 
 # "Upham's Corner - Ramsey St at Dudley St" are located at the same place!
dict_3 = {'Station': "Upham's Corner - Ramsey St at Dudley St", 'Latitude': 42.365942, 'Longitude': -71.060515}
dict_4 = {'Station': "Beacon St at Arlington St", 'Latitude': 42.35554932336134, 'Longitude': -71.07284188270569}
dict_5 = {'Station': "Boylston St at Washington St", 'Latitude': 42.352409, 'Longitude': -71.062679}
dict_6 = {'Station': "Central Square - East Boston", 'Latitude': 42.373312125824704, 'Longitude': -71.0410200806291}
dict_7 = {'Station': "Commonwealth Ave at Buick St", 'Latitude': 42.35062177153799, 'Longitude': -71.11288218766276}

geo_location_3 = pd.DataFrame([dict_1, dict_2, dict_3, dict_4, dict_5, dict_6, dict_7])
geo_location_3

In [ ]:
geo_location = pd.concat([geo_location, geo_location_3])
geo_location.head(5)

In [ ]:
merged_df = merge()
merged_df.head(5)

In [ ]:
getMissingStations()

**Alright, we've almost got it, only these strange stations with names `8D OPS 01` and `8D OPS 03` left.**
<br> These stations have cryptic names and also clearly-incorrect coordinates (e. g. (0,0) for ``8D OPS 01``). There were no metadata explaining these stations at BlueBike. Let's look at every entry containing one of the station:

In [ ]:
def checkName(name: str) -> bool:
    return name[:6]  == "8D OPS" #check if first 6 characters match the generic prefix "8D OPS"

cryptic_stations = merged_df[(merged_df["start_station_name"].apply(lambda x: checkName(x))) | 
                             (merged_df["end_station_name"].apply(lambda x: checkName(x)))]
cryptic_stations

It seems that these stations are **not public rental stations** and that these "trips" refect inventory movements or other internal events not related to customer. This also seems rational as they only were used 2 times ``8D OPS 01`` and 4 times for ``8D OPS 03`` as you can see above and also never started or ended at a "real" station. Thus, we drop every row in the original dataframe which has one of these names in their `start_station_name` or `end_station_name`.

In [ ]:
merged_df.drop(index=cryptic_stations.index, inplace=True)

In [ ]:
missing_stations = merged_df[merged_df.isnull().any(1)]
missing_stations_grouped = missing_stations.groupby("start_station_name").nunique()
missing_stations_grouped

In [ ]:
len(missing_stations_grouped)

##### Yeah, now every station has it's own coordinates! 🎉🎉

### Fourth Step: Visualize the geographical position of the stations

In [ ]:
# folium needs the coordinates in one variable, so putting the values together.

# We're rounding the corrdinates to 4 digits in order to remove some inaccuracies while getting the data.
merged_df["Coordinates_Start"] = list(zip(merged_df["Latitude_Start"].round(4),merged_df["Longitude_Start"].round(4))) 
merged_df["Coordinates_End"] = list(zip(merged_df["Latitude_End"].round(4),merged_df["Longitude_End"].round(4))) 
merged_df.head(5)

##### Stations that are frequently used to start a trip

In [ ]:
heat_map = folium.Map(location=(42.36, -71.071), tiles='Stamen Toner',zoom_start=12, control_scale=True, max_zoom=20)

# add heat map
heat_map.add_child(plugins.HeatMap(merged_df["Coordinates_Start"], radius=20))

heat_map

##### Stations that are frequently used to end a trip

In [ ]:
heat_map = folium.Map(location=(42.36, -71.071), tiles='Stamen Toner',zoom_start=12, control_scale=True, max_zoom=20)

# add heat map
heat_map.add_child(plugins.HeatMap(merged_df["Coordinates_End"], radius=20))

heat_map

We see almost no differences, **but some stations** (see e. g. `Hardvard Street`) **at the outside of the city are more often used to rent a bike** (likely people how use the bike to travel back inside the city) **instead of returning a bike**.
_(Note: You have to zoom into both maps, to see the difference)_ 

This could indicate that BlueBikes need to pay attention especially to these outside bike stations in order to initiate inventory movements or provide some incentives (like discounts) for the people who start a trip in the city and bring the bike back e. g. to the station next to `Hardvard Street`.

##### Frequently used stations (no matter whether used as start OR end station)

In [ ]:
#generating a new list holding all coordinates of the trips
def addCoordinatesToList(coordinates, list):
    list.append(coordinates)

list_with_start_and_end_coordinates = []
merged_df["Coordinates_Start"].apply(lambda x: addCoordinatesToList(x, list_with_start_and_end_coordinates))
merged_df["Coordinates_End"].apply(lambda x: addCoordinatesToList(x, list_with_start_and_end_coordinates))
list_with_start_and_end_coordinates[0:5] #holds a list with all coordaintes of the data table

In [ ]:
heat_map = folium.Map(location=(42.36, -71.071), tiles='Stamen Toner',zoom_start=12, control_scale=True, max_zoom=20)

# add heat map
heat_map.add_child(plugins.HeatMap(list_with_start_and_end_coordinates, radius=20))

heat_map

In conclusion, this map provides an total overview of the activity level of the different stations. This can be useful to determine stations, that are not used frequently and thus, could be removed.

## KPI 1: Estimated Revenue 

Although, we only have roughly fitting data for calculating the revenue, we decided to keep track on it in order to be able to show one of the most important KPIs for business practionioners: **The renvenue**!

As we have the dataset for 2016, we needed to do some more research on the pricing in 2016. We used the `Wayback Machine` in order to get the old website of BlueBikes (which were named at this time "hubway") and their pricings back in 2016. This revealed the following pricing:

![Fees](pricing_membership_overtimefees_960px.png)

_If the bike has not been returned and correctly docked at a station after 24 hours, the bike is considered stolen and a fee of $1,000 will be charged to your credit card._

Source: [https://web.archive.org/web/20160331184858/http://www.thehubway.com/pricing](https://web.archive.org/web/20160331184858/http://www.thehubway.com/pricing)

So the following equation holds:
$$Total Revenue = Membership Fees + Overtime Fees $$

As we do not have any data about active members, we unfortunately cannot calcualte the total revenue. **We instead focus on calculating the additional income generated by the `Overtime Fees`.**

In [ ]:
boston["start_time"] = pd.to_datetime(boston["start_time"])
boston["end_time"] = pd.to_datetime(boston["end_time"])
boston["trip_time"] = boston["end_time"] - boston["start_time"]
boston["day"] = boston["start_time"].apply(lambda ts: ts.date())
boston["month"] = boston["start_time"].apply(lambda ts: ts.month)
boston["hour"] = boston["start_time"].apply(lambda ts: ts.hour)
boston.head(5)

As the dataset is quite huge and the algorithm has to calcualte the fee manually for each trip, it takes up to 5 minutes...

In [ ]:
def getAmountOfHalfHour(trip_time):
    anz = trip_time / pd.Timedelta('30min')
    return math.floor(anz) #round down: 1.5 becomes 1 as next full 1/2 hour not reached.

def getOvertimeFeeForTrip(row):
    trip_time = row["trip_time"]
    user_type = row["user_type"]
    
    overtime_fee = 0
    
    if trip_time < pd.Timedelta('30min'):
        overtime_fee = 0
    elif trip_time < pd.Timedelta('60min'):
        overtime_fee = 2
    elif trip_time < pd.Timedelta('90min'):
        overtime_fee = 4
    else:
        overtime_fee = 4
        # every 1/2 hour +8$ are applied
        count = getAmountOfHalfHour(trip_time)
        count = count - 3 #first 90mins are already included by the 4$ above
        overtime_fee += 8*count
        
    #applying 25% discount for members
    if user_type == "Subscriber":
        overtime_fee -= overtime_fee*0.25
    
    return overtime_fee
        
    
boston["overtime_fee"] = boston.apply(getOvertimeFeeForTrip, axis=1) #use axis=1 to pass the series object row-wise
boston.head(5)

In [ ]:
# delete outlier
indexes_to_delete = boston[boston["overtime_fee"] > 300].index
boston.drop(index=indexes_to_delete, inplace=True)

In [ ]:
total_revenue_per_day = boston[["day", "overtime_fee"]].groupby("day").sum() # Getting the total revenue per day
total_revenue_per_day.head(5)

In [ ]:
# Plot
fig,ax = plt.subplots(figsize=(16,9)) 

ax.plot(total_revenue_per_day["overtime_fee"])
ax.set_title("Revenue through overtime fees per day",fontsize=16)

plt.show()

We can see a **seasonal trend**: In summer months, people tend to use the bikes longer then in winter months.

In [ ]:
fig,ax = plt.subplots(figsize=(16,9)) 
sns.boxplot(x="hour",y="overtime_fee",data=boston,palette="mako") 
plt.show()

It seems, that most of the time the customer do not overrun their time greatly.

In [ ]:
fig,ax = plt.subplots(figsize=(16,9)) 
sns.distplot(boston["overtime_fee"], kde=True)
plt.show()

Yeah, let's remove the outliers for this visualization and only look at the height of fee if any overtime fee was charged.

In [ ]:
fig,ax = plt.subplots(figsize=(16,9)) 
df = boston[(boston["overtime_fee"] > 0 ) & (boston["overtime_fee"] < 80)]
sns.distplot(df["overtime_fee"], kde=False)
plt.show()

We see, that customers try most of the time to not exceed the inlcuded free 30mins. However, lot's of people exceed 30mins by a short amount of time and thus are charged by 1.5 US-Dollar or 2 US-Dollar (depending on whether they are customer or not). 
Only rarely, the time limit of 30mins gets more greatly.

In [ ]:
def getTimePassedInSec(datetime):
    return datetime.total_seconds()

fig,ax = plt.subplots(figsize=(16,9)) 

boston_new = boston.copy()
boston_new["trip_time"] = boston_new["trip_time"].apply(lambda x: getTimePassedInSec(x))


sns.scatterplot(ax=ax, x="trip_time", y="overtime_fee", 
                data=boston_new, hue="user_type")

ax.set_xlabel("Trip duration in seconds")
ax.set_ylabel("Overtime fees charged in $")
ax.set_title("Overtime fees per trip length", fontsize=14)
ax.legend(fontsize=12)

plt.show()

It seems that subscribers tend to do longer trips than customers (propably due to the 25% discount the subscribers get)

## KPI 2: Most Popular Stations

## KPI 2: Popular trip routes

In [ ]:
start_ids_per_day_group = boston[["day", "start_station_id", "start_time"]].groupby(["day", "start_station_id"]).count()

In [ ]:
end_ids_per_day_group = boston[["day", "end_station_id", "start_time"]].groupby(["day", "end_station_id"]).count()

In [ ]:
total_ids_per_day_group = start_ids_per_day_group + end_ids_per_day_group
total_ids_per_day_group.rename(columns={'start_time':'Times this trip appears in dataset'}, inplace=True)
total_ids_per_day_group

In [ ]:
popular_trips_group = boston.groupby(["start_station_id", "end_station_id"]).size().unstack(level=0)

#popular_trips_group.head(500)

#every NaN means that in whole 2016 there was no trip to it, so replace NaN with zero.

In [ ]:
for i in popular_trips_group.index:
    print(i)

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(20, 15))
sns.heatmap(popular_trips_group, robust=True)
plt.show()
# TO DO beschrifte jede Linie

#### This heatmap looks a bit unordner, but it conveys quite useful information

1. We see lots of dark point, but also some red points. These red and light points are especially important, as lots of trips start respective end there. That means that BlueBike has to take care that these stations are working correctly (e. g. no technical errors, always enough bikes available, might even add more docker stations for bikes) because otherwise a malfunction could directly lead to customer dissatification.
1. Wee see that in general almost every station is used. That means that we do not have unnecessary stations that only create costs but no profit. However, some white spaces for the stations `211` or `214` could indicate that they are not often used. However, further investigation is needed.
1. Stations `153` and `158` are huge outliners: They are only used by each other which is a quite unusual pattern (see the drawn through white line at the corresponding station IDs). Further investigation reveals that that are the technical 8D OPS stations that are not used publicly.

In [ ]:
# Getting the most popular routes (used more than 1000 times)
popular_trips_group = boston.groupby(["start_station_id", "end_station_id"]).size()
popular_trips = pd.DataFrame(popular_trips_group[popular_trips_group > 1000], columns=["count"])
popular_trips.reset_index(inplace=True)
popular_trips.head(5)

In [ ]:
# Create normalized dataset
max = popular_trips["count"].max()
min = popular_trips["count"].min()
popular_trips["norm_value"] = popular_trips["count"].apply(lambda x: (x - min) / (max - min) )
popular_trips.head(5)

In [ ]:
# helper methods for path visualization
def getNameFromStationId(id):
    df = merged_df[merged_df["start_station_id"]==id]
    name = df.iloc[0]["start_station_name"]
    return name
    
    
def getCoordinatesFromStationId(id):
    df = merged_df[merged_df["start_station_id"]==id]
    
    #as some routes overlap, we use a every small jitter in order to be able to display overlapping lines.
    jitter = random.uniform(-0.0001,+0.0001)
    lat = df.iloc[0]["Latitude_Start"] + jitter
    long = df.iloc[0]["Longitude_Start"] + jitter
    return [lat, long]

In [ ]:
ordered_color_dict = col.TABLEAU_COLORS #use pre-defined colors

colors = []
for key, value in ordered_color_dict.items():
   colors.append(value)


map = folium.Map(location=(42.38, -71.1),  tiles='CartoDB positron', zoom_start=13, control_scale=True, max_zoom=20)

# draw trajectory
for index, (start_station_id, end_station_id, count, norm_value) in popular_trips.iterrows():
    if start_station_id != end_station_id:
        coordinates_list = []
        coordinates_list.append(getCoordinatesFromStationId(start_station_id))
        coordinates_list.append(getCoordinatesFromStationId(end_station_id))
        displayMessage = "Trip from station " + getNameFromStationId(start_station_id) + " to station " + getNameFromStationId(end_station_id)
        folium.PolyLine(coordinates_list, color=colors[index%len(colors)], popup=displayMessage, weight=20*norm_value, opacity=0.8).add_to(map)
    else:
        lat = getCoordinatesFromStationId(start_station_id)[0]
        long = getCoordinatesFromStationId(start_station_id)[1]
        folium.CircleMarker(location=(lat, long), popup="Trip started and ended at " + getNameFromStationId(start_station_id), 
                            radius=20*norm_value, opcity=0.8, color=colors[index%len(colors)]).add_to(map)
            
    
map

We see a map with the most popular trips in 2016. Every trip which was used in 2016 more than 1000 times shown as a line on the map. To get the exact nameing of the start and end station, just click on any line. The more often a trip was used, the more thicker the line is. Colors do not have a special meaning, the are just used to differentiate the trips. Trips that started and ended at the same station, are marked as circle.

**What insights do we obtain?**
1. The most popular trips took place in the inner city. Bikes are escpecially used to travel over the `Massachusetts Ave` bridge.
1. The stations in the inner city are used very often, so the decision of BlueBikes to build lots of stations in the inner city to enable the customers with a more flexible routing in the city was right.
1. Outside the city, there are two places that are used very frequently, too and should be carefully monitored by BlueBikes: At `Davis Square` and `Financial district`.
1. Two stations are **used frequently as start and end point**: `MIT at Mass Ave / Amherst St` and `The Esplanade - Beacon St. at Arlington St.`. Further investigation could determine whether people might miss another returning station and thus, are forced to return back to their start station as every other station would be too far away. This could indicate a potential lack of stations in the bike sharing infrastructure.